In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/train.csv", low_memory=False)
test = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/test.csv", low_memory=False)
train_df.info(memory_usage="deep")

We can see that all the columns have non null values so no worries

In [ ]:
train_df.head(10)

In [ ]:
train_df["target"].unique() # There are total of 9 classes

In [ ]:
Summary = pd.DataFrame(train_df.dtypes, columns=['Dtype'])
Summary["max"] = train_df.max()
Summary["min"] = train_df.min()
Summary["Null"] = train_df.isnull().sum() # to get null values
Summary["First"] = train_df.iloc[0] # to get first value
Summary["Second"] = train_df.iloc[1] # to get second value
Summary


Since the values are at diff scales, we need to deal with scaling.

In [ ]:
train_df = train_df.drop(["id"], axis = 1)

In [ ]:
target = train_df["target"]

In [ ]:
train_df = train_df.drop(["target"], axis = 1)

We can see that there are 200,000 values in here which are alot

In [ ]:
test = test.drop(["id"], axis = 1)

In [ ]:
values, counts = np.unique(target, return_counts=True)

In [ ]:
values

In [ ]:
res = (counts/len(target)) * 100
print(res)

We can see that the values are not evenly distributed so we need to split the data carefully

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(train_df, target, test_size = 0.1, stratify=target, random_state = 123)

Checking whether straification worked?

In [ ]:
values, counts = np.unique(y_train, return_counts=True)
print(values)
print(counts)
res = (counts/len(y_train)) * 100
print(res)

In [ ]:
values, counts = np.unique(y_test, return_counts=True)
print(values)
print(counts)
res = (counts/len(y_test)) * 100
print(res)

In [ ]:
enc = LabelEncoder()
y_train = enc.fit_transform(y_train)
y_test = enc.transform(y_test)    
# test = enc.transform(test)

In [ ]:
train_df.columns

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
test = scaler.transform(test)

In [ ]:
# model = LogisticRegression(random_state=123 )
import lightgbm as lgb
# model = RandomForestClassifier(n_estimators=500, max_leaf_nodes=3, max_depth=10, random_state=123)
# model = RandomForestClassifier(n_estimators=300, max_depth=15, n_jobs=-1, random_state=123)
# model.fit(X_train, y_train)
model=lgb.LGBMClassifier()

# clf.fit(X_train_valid, y_train_valid)

In [ ]:
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_test)

In [ ]:
print(y_pred)
print(y_test)
values, counts = np.unique(y_pred, return_counts=True)
print(values)
print(counts)

In [ ]:
from sklearn.metrics import log_loss
val = log_loss(y_test,y_pred)
print(val)
# 1.8307023065814374 for LR
# 4.002083813135203 for RC
# 1.8306874700003668 for parameterized RC
# 1.7616033784491318 for more parametrized RC
# 1.755212422361672 for lightbgm

In [ ]:
sub = pd.read_csv("/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv")
y_res = model.predict_proba(test)

In [ ]:
y_res.shape

In [ ]:
sub.head(10)
predictions = pd.DataFrame()
predictions["id"] = sub["id"]
predictions = pd.concat([predictions, pd.DataFrame(y_res, columns=["Class_" + str(x) for x in np.arange(1, 10, 1)])], axis=1)
predictions.to_csv('submission.csv', index=False, header=predictions.columns)
predictions.head()